In [110]:
import os
import subprocess
from IPython.display import display, HTML, Image
import io
from operator import itemgetter


import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.tree import export_graphviz
from sklearn import tree
from sklearn import metrics
from sklearn import tree
from sklearn import svm
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn import ensemble
from sklearn import linear_model
from sklearn import neighbors

from xgboost import XGBClassifier

%matplotlib inline
%qtconsole

### 1. Load and explore	the	MNIST fashion training dataset

In [111]:
train_data = pd.read_csv('fashion-mnist_train.csv')
test_data = pd.read_csv('fashion-mnist_test.csv')
dataset.head()

,label,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
0,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,9,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,6,0,0,0,0,0,0,0,5,0,...,0,0,0,30,43,0,0,0,0,0
3,0,0,0,0,1,2,0,0,0,0,...,3,0,0,0,0,1,0,0,0,0
4,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [112]:
dataset['label'].value_counts()

9    6000
8    6000
7    6000
6    6000
5    6000
4    6000
3    6000
2    6000
1    6000
0    6000
Name: label, dtype: int64

### 2. Divide the MNIST	fashion	training	dataset	into a	train	and	validation	datasets	

In [113]:
X=train_data.loc[:,train_data.columns != 'label']
Y=train_data['label']
x_test=test_data.loc[:,test_data.columns != 'label']
y_test=test_data['label']

In [114]:
X_train, X_valid, y_train, y_valid \
    = train_test_split(X,Y,random_state=0, \
                                        train_size = 0.7)

/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


### 3. Build a	decision tree model	to	perform	the	MNIST	fashion	classification	task

In [115]:
my_tree = \
    tree.DecisionTreeClassifier(criterion="entropy")
my_tree.fit(X_train,y_train)

DecisionTreeClassifier(class_weight=None, criterion='entropy', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [116]:
my_tree.fit(X_train, y_train)
my_tree.classes_

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

### 4. Evaluate	the	performance	of	this tree on the	MNIST fashion	validation	set	created previously

In [117]:
y_pred = my_tree.predict(X_valid,)

# Print performance details
accuracy = \
metrics.accuracy_score(y_valid, y_pred) # , normalize=True, sample_weight=None
print("Accuracy: " +  str(accuracy))

print(metrics.classification_report(y_valid, y_pred))

# Print confusion matrix
print(metrics.confusion_matrix(y_valid, y_pred))

# Print nicer homemade confusion matrix
print("Confusion Matrix")
pd.crosstab(y_valid, y_pred, rownames=['True'], colnames=['Predicted'], margins=True)

Accuracy: 0.8073888888888889
             precision    recall  f1-score   support

          0       0.75      0.76      0.76      1799
          1       0.94      0.94      0.94      1843
          2       0.70      0.70      0.70      1808
          3       0.81      0.80      0.80      1849
          4       0.70      0.70      0.70      1765
          5       0.92      0.90      0.91      1786
          6       0.56      0.55      0.56      1787
          7       0.88      0.88      0.88      1789
          8       0.91      0.92      0.91      1795
          9       0.90      0.91      0.91      1779

avg / total       0.81      0.81      0.81     18000

[[1371   10   35   85   17    3  252    0   26    0]
 [  13 1735   10   52   11    1   17    0    4    0]
 [  37    4 1270   30  208    1  238    0   18    2]
 [  82   62   39 1475   98    1   68    0   20    4]
 [  13    9  222   98 1238    0  169    0   16    0]
 [   1    2    2    2    3 1606    1   94   25   50]
 [ 280   16  2

Predicted,0,1,2,3,4,5,6,7,8,9,All
True,,,,,,,,,,,
0,1371,10,35,85,17,3,252,0,26,0,1799
1,13,1735,10,52,11,1,17,0,4,0,1843
2,37,4,1270,30,208,1,238,0,18,2,1808
3,82,62,39,1475,98,1,68,0,20,4,1849
4,13,9,222,98,1238,0,169,0,16,0,1765
5,1,2,2,2,3,1606,1,94,25,50,1786
6,280,16,216,64,180,3,986,0,41,1,1787
7,0,0,0,0,0,85,0,1582,7,115,1789
8,22,4,14,24,20,14,32,12,1644,9,1795


### 5. Use	a grid search	to	tune the hyperparaemters for this model

In [118]:
param_grid ={'criterion': ['gini', "entropy"], \
             'max_depth': list(range(3, 20, 3)), \
             'min_samples_split': [50] }

# Perform the search
my_tuned_tree = GridSearchCV(tree.DecisionTreeClassifier(), \
                                param_grid, cv=2, verbose = 0, \
                            return_train_score=True)
my_tuned_tree.fit(X, Y)

# Print details
print("Best parameters set found on development set:")
display(my_tuned_tree.best_params_)
display(my_tuned_tree.best_score_)
display(my_tuned_tree.cv_results_)

Best parameters set found on development set:


{'criterion': 'entropy', 'max_depth': 12, 'min_samples_split': 50}

0.80365

{'mean_fit_time': array([ 3.15410817,  5.19033194,  7.62739003, 10.29874456, 12.19825852,
        13.5842576 ,  2.7216996 ,  6.04979241,  9.77922451, 12.72304106,
        14.14198291, 14.3395927 ]),
 'mean_score_time': array([0.08238792, 0.06303656, 0.0610075 , 0.06730294, 0.0702889 ,
        0.0700295 , 0.05395794, 0.0617435 , 0.06231141, 0.06491196,
        0.06596446, 0.0653075 ]),
 'mean_test_score': array([0.4949    , 0.72925   , 0.78908333, 0.80165   , 0.79561667,
        0.79116667, 0.52301667, 0.74883333, 0.79293333, 0.80365   ,
        0.80203333, 0.80038333]),
 'mean_train_score': array([0.49688333, 0.73661667, 0.81983333, 0.86335   , 0.88166667,
        0.88701667, 0.5249    , 0.75673333, 0.82681667, 0.86516667,
        0.87883333, 0.88101667]),
 'param_criterion': masked_array(data=['gini', 'gini', 'gini', 'gini', 'gini', 'gini',
                    'entropy', 'entropy', 'entropy', 'entropy', 'entropy',
                    'entropy'],
              mask=[False, False, False

In [119]:
y_pred = my_tuned_tree.predict(x_test)

# Print performance details
accuracy = metrics.accuracy_score(y_test, y_pred) # , normalize=True, sample_weight=None
print("Accuracy: " +  str(accuracy))
print(metrics.classification_report(y_test, y_pred))

# Print confusion matrix
print("Confusion Matrix")
pd.crosstab(y_test, y_pred, rownames=['True'], colnames=['Predicted'], margins=True)

Accuracy: 0.8179
             precision    recall  f1-score   support

          0       0.74      0.80      0.77      1000
          1       0.95      0.95      0.95      1000
          2       0.72      0.71      0.72      1000
          3       0.84      0.85      0.84      1000
          4       0.71      0.75      0.73      1000
          5       0.89      0.89      0.89      1000
          6       0.61      0.52      0.56      1000
          7       0.85      0.89      0.87      1000
          8       0.93      0.92      0.92      1000
          9       0.91      0.89      0.90      1000

avg / total       0.82      0.82      0.82     10000

Confusion Matrix


Predicted,0,1,2,3,4,5,6,7,8,9,All
True,,,,,,,,,,,
0,801,5,11,32,4,4,133,0,10,0,1000
1,10,952,4,23,3,2,3,0,3,0,1000
2,16,2,714,14,147,0,97,0,9,1,1000
3,42,30,12,846,37,3,23,0,5,2,1000
4,3,3,127,59,750,0,52,0,5,1,1000
5,3,3,0,1,0,892,0,73,11,17,1000
6,195,8,113,31,105,0,524,0,23,1,1000
7,0,0,0,0,0,56,0,888,2,54,1000
8,7,2,7,6,4,15,23,8,921,7,1000


### 6.  Experiment	with	different	machine	learning	algorithms	to	see	if	you	can	build	a	better	model	(try	ensembles,	bagging,	boosting,	random	forests,	…)	

In [127]:
# Do the same job with random forests
my_model = ensemble.RandomForestClassifier(n_estimators=300, \
                                           max_features = 3,\
                                           min_samples_split=200)
my_model.fit(X,Y)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features=3, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=200,
            min_weight_fraction_leaf=0.0, n_estimators=300, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [128]:
y_pred = my_model.predict(x_test)

metrics.accuracy_score(y_test, y_pred) # , normalize=True, sample_weight=None
print("Accuracy: " +  str(accuracy))

# Print performance details
print(metrics.classification_report(y_test, y_pred))

# Print confusion matrix
print("Confusion Matrix")
pd.crosstab(y_test, y_pred, rownames=['True'], colnames=['Predicted'], margins=True)

Accuracy: 0.8179
             precision    recall  f1-score   support

          0       0.74      0.81      0.78      1000
          1       0.99      0.94      0.96      1000
          2       0.73      0.77      0.75      1000
          3       0.78      0.92      0.84      1000
          4       0.73      0.77      0.75      1000
          5       0.96      0.91      0.93      1000
          6       0.73      0.44      0.55      1000
          7       0.88      0.88      0.88      1000
          8       0.91      0.97      0.94      1000
          9       0.89      0.93      0.91      1000

avg / total       0.83      0.83      0.83     10000

Confusion Matrix


Predicted,0,1,2,3,4,5,6,7,8,9,All
True,,,,,,,,,,,
0,812,0,23,91,6,1,43,0,24,0,1000
1,2,942,21,30,2,1,2,0,0,0,1000
2,9,1,771,8,145,1,50,0,15,0,1000
3,23,7,11,918,25,0,14,0,2,0,1000
4,0,1,101,69,773,1,48,0,7,0,1000
5,0,0,0,0,0,909,0,61,8,22,1000
6,246,2,125,55,101,0,439,0,32,0,1000
7,0,0,0,0,0,28,0,876,0,96,1000
8,1,1,11,4,3,2,3,4,970,1,1000


In [126]:
my_model = ensemble.BaggingClassifier(base_estimator = tree.DecisionTreeClassifier(criterion="entropy", min_samples_leaf = 50), \
                                      n_estimators=10)
my_model.fit(X_train,y_train)

BaggingClassifier(base_estimator=DecisionTreeClassifier(class_weight=None, criterion='entropy', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=50, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best'),
         bootstrap=True, bootstrap_features=False, max_features=1.0,
         max_samples=1.0, n_estimators=10, n_jobs=1, oob_score=False,
         random_state=None, verbose=0, warm_start=False)

In [131]:
y_pred = my_model.predict(X_valid)

metrics.accuracy_score(y_valid, y_pred) # , normalize=True, sample_weight=None
print("Accuracy: " +  str(accuracy))

# Print performance details
print(metrics.classification_report(y_valid, y_pred))

# Print confusion matrix
print("Confusion Matrix")
pd.crosstab(y_valid, y_pred, rownames=['True'], colnames=['Predicted'], margins=True)

Accuracy: 0.8179
             precision    recall  f1-score   support

          0       0.78      0.84      0.81      1799
          1       1.00      0.94      0.97      1843
          2       0.75      0.82      0.78      1808
          3       0.79      0.93      0.86      1849
          4       0.75      0.79      0.77      1765
          5       0.97      0.94      0.95      1786
          6       0.77      0.47      0.58      1787
          7       0.92      0.92      0.92      1789
          8       0.92      0.97      0.94      1795
          9       0.92      0.95      0.94      1779

avg / total       0.86      0.86      0.85     18000

Confusion Matrix


Predicted,0,1,2,3,4,5,6,7,8,9,All
True,,,,,,,,,,,
0,1508,1,39,136,10,1,65,0,39,0,1799
1,1,1732,39,60,4,0,6,0,1,0,1843
2,4,0,1477,15,219,0,69,0,24,0,1808
3,41,2,9,1719,35,0,40,0,3,0,1849
4,7,1,142,137,1399,0,64,0,15,0,1765
5,0,0,0,0,0,1674,0,77,9,26,1786
6,377,2,236,89,184,1,836,0,62,0,1787
7,0,0,0,0,0,31,0,1643,3,112,1789
8,1,0,20,10,8,3,11,4,1736,2,1795


In [132]:
# Do the same job with random forests
my_model = ensemble.AdaBoostClassifier(base_estimator = tree.DecisionTreeClassifier(criterion="entropy", min_samples_leaf = 50), \
                                       n_estimators=10)
my_model.fit(X_train,y_train)

AdaBoostClassifier(algorithm='SAMME.R',
          base_estimator=DecisionTreeClassifier(class_weight=None, criterion='entropy', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=50, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best'),
          learning_rate=1.0, n_estimators=10, random_state=None)

In [134]:
y_pred = my_model.predict(x_test)

metrics.accuracy_score(y_test, y_pred) # , normalize=True, sample_weight=None
print("Accuracy: " +  str(accuracy))

# Print performance details
print(metrics.classification_report(y_test, y_pred))

# Print confusion matrix
print("Confusion Matrix")
pd.crosstab(y_test, y_pred, rownames=['True'], colnames=['Predicted'], margins=True)

Accuracy: 0.8179
             precision    recall  f1-score   support

          0       0.77      0.70      0.73      1000
          1       0.99      0.95      0.97      1000
          2       0.68      0.66      0.67      1000
          3       0.83      0.87      0.85      1000
          4       0.69      0.66      0.67      1000
          5       0.91      0.89      0.90      1000
          6       0.46      0.55      0.50      1000
          7       0.85      0.85      0.85      1000
          8       0.96      0.93      0.94      1000
          9       0.90      0.89      0.90      1000

avg / total       0.80      0.79      0.80     10000

Confusion Matrix


Predicted,0,1,2,3,4,5,6,7,8,9,All
True,,,,,,,,,,,
0,698,0,16,53,4,0,220,1,8,0,1000
1,8,947,6,24,5,0,10,0,0,0,1000
2,12,0,659,11,129,0,185,0,4,0,1000
3,40,6,10,867,33,0,41,0,3,0,1000
4,2,1,153,39,657,0,148,0,0,0,1000
5,0,0,0,0,0,894,3,76,10,17,1000
6,149,2,119,45,126,0,546,0,13,0,1000
7,0,0,0,0,0,62,0,855,1,82,1000
8,1,0,9,2,4,3,46,1,931,3,1000


### 7. Experiment	with feature engineering that you could	perform	in	order to	do	better	at	this problem.

In [135]:
param_grid ={'criterion': ['gini', "entropy"], \
             'max_depth': list(range(3, 20, 3)), \
             'min_samples_split': [50] }

# Perform the search
my_tuned_tree = GridSearchCV(ensemble.RandomForestClassifier(n_estimators=300, \
                                           max_features = 3,\
                                           min_samples_split=200), \
                                param_grid, cv=2, verbose = 0, \
                            return_train_score=True)
my_tuned_tree.fit(X, Y)

# Print details
print("Best parameters set found on development set:")
display(my_tuned_tree.best_params_)
display(my_tuned_tree.best_score_)
display(my_tuned_tree.cv_results_)

Best parameters set found on development set:


{'criterion': 'gini', 'max_depth': 18, 'min_samples_split': 50}

0.8415

{'mean_fit_time': array([ 3.80901527,  5.96616662, 12.49338007, 13.16856444, 12.09217846,
        14.17892456,  4.15811205,  7.0224365 , 10.56056249, 13.48906243,
        15.24047601, 16.0542264 ]),
 'mean_score_time': array([1.01904464, 1.36805987, 3.32901597, 2.9130646 , 3.65599442,
        3.78156447, 1.02108145, 1.38761389, 2.00294566, 2.6591661 ,
        3.08480012, 3.29658449]),
 'mean_test_score': array([0.69465   , 0.7824    , 0.80831667, 0.82688333, 0.83633333,
        0.8415    , 0.66701667, 0.78186667, 0.80998333, 0.82808333,
        0.83738333, 0.84078333]),
 'mean_train_score': array([0.69531667, 0.78861667, 0.82458333, 0.85733333, 0.87836667,
        0.89098333, 0.67      , 0.7887    , 0.82568333, 0.8561    ,
        0.87723333, 0.88746667]),
 'param_criterion': masked_array(data=['gini', 'gini', 'gini', 'gini', 'gini', 'gini',
                    'entropy', 'entropy', 'entropy', 'entropy', 'entropy',
                    'entropy'],
              mask=[False, False, False

In [136]:
y_pred = my_tuned_tree.predict(x_test)

# Print performance details
accuracy = metrics.accuracy_score(y_test, y_pred) # , normalize=True, sample_weight=None
print("Accuracy: " +  str(accuracy))
print(metrics.classification_report(y_test, y_pred))

# Print confusion matrix
print("Confusion Matrix")
pd.crosstab(y_test, y_pred, rownames=['True'], colnames=['Predicted'], margins=True)

Accuracy: 0.851
             precision    recall  f1-score   support

          0       0.76      0.82      0.79      1000
          1       0.99      0.95      0.97      1000
          2       0.77      0.78      0.77      1000
          3       0.82      0.92      0.87      1000
          4       0.75      0.82      0.78      1000
          5       0.97      0.92      0.94      1000
          6       0.71      0.50      0.58      1000
          7       0.89      0.90      0.89      1000
          8       0.93      0.97      0.95      1000
          9       0.90      0.94      0.92      1000

avg / total       0.85      0.85      0.85     10000

Confusion Matrix


Predicted,0,1,2,3,4,5,6,7,8,9,All
True,,,,,,,,,,,
0,820,0,16,74,4,1,67,0,18,0,1000
1,1,955,14,25,2,1,2,0,0,0,1000
2,8,1,781,8,137,0,52,0,13,0,1000
3,24,6,4,916,24,0,25,0,1,0,1000
4,0,1,80,49,816,0,50,0,4,0,1000
5,0,0,0,0,0,921,0,55,6,18,1000
6,227,1,114,40,98,0,496,0,24,0,1000
7,0,0,0,0,0,20,0,898,0,82,1000
8,1,1,10,2,4,2,4,4,971,1,1000
